In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install xlrd


In [ ]:
import re

def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=xlsx'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url



### Exercise 1: Load and examine a superstore sales data from an Excel file

In [ ]:
df = pd.read_excel(convert_google_sheet_url("https://docs.google.com/spreadsheets/d/1_p_pS25SlGwDF3t9vd8PRXnF9LKEjN85/edit#gid=1719135658"),sheet_name='Orders')

In [ ]:
df.head(10)

In [ ]:
df.drop('Row ID',axis=1,inplace=True)

In [ ]:
df.shape

### Exercise 2: Subsetting the DataFrame

In [ ]:
df_subset = df.loc[[i for i in range(5,10)],['Customer ID','Customer Name','City','Postal Code','Sales']]
df_subset

### Exercise 3: An example use case – determining statistics on sales and profit for records 100-199

In [ ]:
df_subset = df.loc[[i for i in range(100,200)],['Sales','Profit']]

In [ ]:
df_subset.describe()

In [ ]:
df_subset.plot.box()
plt.title("Boxplot of sales and profit",fontsize=15)
plt.ylim(0,500)
plt.grid(True)
plt.show()

### Exercise 4: A useful function – unique

In [ ]:
df['State'].unique()

In [ ]:
df['State'].nunique()

In [ ]:
df['Country'].unique()

In [ ]:
df.drop('Country',axis=1,inplace=True)

### Exercise 5: Conditional Selection and Boolean Filtering

In [ ]:
df_subset = df.loc[[i for i in range (10)],['Ship Mode','State','Sales']]
df_subset

In [ ]:
df_subset[df_subset['Sales']>100]

In [ ]:
df_subset[(df_subset['State']!='California') & (df_subset['Sales']>100)]

### Exercise 6: Setting and re-setting index

In [ ]:
matrix_data = np.matrix('22,66,140;42,70,148;30,62,125;35,68,160;25,62,152')
row_labels = ['A','B','C','D','E']
column_headings = ['Age', 'Height', 'Weight']

df1 = pd.DataFrame(data=matrix_data, index=row_labels, columns=column_headings)
print("\nThe DataFrame\n",'-'*25, sep='')
print(df1)
print("\nAfter resetting index\n",'-'*35, sep='')
print(df1.reset_index())
print("\nAfter resetting index with 'drop' option TRUE\n",'-'*45, sep='')
print(df1.reset_index(drop=True))
print("\nAdding a new column 'Profession'\n",'-'*45, sep='')
df1['Profession'] = "Student Teacher Engineer Doctor Nurse".split()
print(df1)
print("\nSetting 'Profession' column as index\n",'-'*45, sep='')
print (df1.set_index('Profession'))

### Exercise 7: GroupBy method

In [ ]:
df_subset = df.loc[[i for i in range (10)],['Ship Mode','State','Sales']]
df_subset

In [ ]:
byState = df_subset.groupby('State')

In [ ]:
byState

In [ ]:
print("\nGrouping by 'State' column and listing mean sales\n",'-'*50, sep='')
print(byState.mean())

In [ ]:
print("\nGrouping by 'State' column and listing total sum of sales\n",'-'*50, sep='')
print(byState.sum())

In [ ]:
print(pd.DataFrame(df_subset.groupby('State').describe().loc['California']).transpose())

In [ ]:
df_subset.groupby('Ship Mode').describe().loc[['Second Class','Standard Class']]

In [ ]:
pd.DataFrame(byState.describe().loc['California'])

In [ ]:
byStateCity=df.groupby(['State','City'])

In [ ]:
byStateCity.describe()['Sales']

### Exercise 8: Missing values in Pandas

In [ ]:
df_missing=pd.read_excel(convert_google_sheet_url("https://docs.google.com/spreadsheets/d/1_p_pS25SlGwDF3t9vd8PRXnF9LKEjN85/edit#gid=18319337"),sheet_name='Missing')

In [ ]:
df_missing

In [ ]:
df_missing.isnull()

In [ ]:
for c in df_missing.columns:
    miss = df_missing[c].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(c,miss))
    else:
        print("{} has NO missing value!".format(c))

### Exercise 9: Filling missing values with `fillna()`

In [ ]:
df_missing.fillna('FILL')

In [ ]:
df_missing[['Customer','Product']].fillna('FILL')

In [ ]:
df_missing['Sales'].fillna(method='ffill')

In [ ]:
df_missing['Sales'].fillna(method='bfill')

In [ ]:
df_missing['Sales'].fillna(df_missing.mean()['Sales'])

### Exercise 10: Dropping missing values with `dropna()`

In [ ]:
df_missing.dropna(axis=0)

In [ ]:
df_missing.dropna(axis=1)

In [ ]:
df_missing.dropna(axis=1,thresh=10)

### Exercise 11: Outlier detection using simple statistical test

In [ ]:
df_sample = df[['Customer Name','State','Sales','Profit']].sample(n=50).copy()

In [ ]:
# Assign a wrong (negative value) in few places
df_sample['Sales'].iloc[5]=-1000.0
df_sample['Sales'].iloc[15]=-500.0

In [ ]:
df_sample.plot.box()
plt.title("Boxplot of sales and profit", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid(True)

### Exercise 12: Concatenation

In [ ]:
df_1 = df[['Customer Name','State','Sales','Profit']].sample(n=4)
df_2 = df[['Customer Name','State','Sales','Profit']].sample(n=4)
df_3 = df[['Customer Name','State','Sales','Profit']].sample(n=4)

In [ ]:
df_1

In [ ]:
df_2

In [ ]:
df_3

In [ ]:
df_cat1 = pd.concat([df_1,df_2,df_3], axis=0)
df_cat1

In [ ]:
df_cat2 = pd.concat([df_1,df_2,df_3], axis=1)
df_cat2

### Exercise 13: Merging by a common key

In [ ]:
df_1=df[['Customer Name','Ship Date','Ship Mode']][0:4]
df_1

In [ ]:
df_2=df[['Customer Name','Product Name','Quantity']][0:4]
df_2

In [ ]:
pd.merge(df_1,df_2,on='Customer Name',how='inner')

In [ ]:
pd.merge(df_1,df_2,on='Customer Name',how='inner').drop_duplicates()

In [ ]:
df_3=df[['Customer Name','Product Name','Quantity']][2:6]
df_3

In [ ]:
pd.merge(df_1,df_3,on='Customer Name',how='inner').drop_duplicates()

In [ ]:
pd.merge(df_1,df_3,on='Customer Name',how='outer').drop_duplicates()

### Exercise 14: Join method

In [ ]:
df_1=df[['Customer Name','Ship Date','Ship Mode']][0:4]
df_1.set_index(['Customer Name'],inplace=True)
df_1

In [ ]:
df_2=df[['Customer Name','Product Name','Quantity']][2:6]
df_2.set_index(['Customer Name'],inplace=True)
df_2

In [ ]:
df_1.join(df_2,how='left').drop_duplicates()

In [ ]:
df_1.join(df_2,how='right').drop_duplicates()

In [ ]:
df_1.join(df_2,how='inner').drop_duplicates()

In [ ]:
df_1.join(df_2,how='outer').drop_duplicates()

## Miscelleneous useful methods

### Exercise 15: Randomized sampling - `sample` method

In [ ]:
df.sample(n=5)

In [ ]:
df.sample(frac=0.001)

In [ ]:
df.sample(frac=0.001,replace=True)

### Exercise 16: Pandas `value_count` method to return unique records

In [ ]:
df['Customer Name'].value_counts()[:10]

### Exercise 17: Pivot table functionality - `pivot_table`

In [ ]:
df_sample=df.sample(n=100)

In [ ]:
df_sample.pivot_table(values=['Sales','Quantity','Profit'],index=['Region','State'],aggfunc='mean')

### Exercise 18: Sorting by particular column

In [ ]:
df_sample=df[['Customer Name','State','Sales','Quantity']].sample(n=15)
df_sample

In [ ]:
df_sample.sort_values(by='Sales')

In [ ]:
df_sample.sort_values(by=['State','Sales'])

### Exercise 19: Flexibility for user-defined function with `apply` method

In [ ]:
def categorize_sales(price):
    if price < 50:
        return "Low"
    elif price < 200:
        return "Medium"
    else:
        return "High"

In [ ]:
df_sample=df[['Customer Name','State','Sales']].sample(n=100)
df_sample.head(10)

In [ ]:
df_sample['Sales Price Category']=df_sample['Sales'].apply(categorize_sales)
df_sample.head(10)

In [ ]:
df_sample['Customer Name Length']=df_sample['Customer Name'].apply(len)
df_sample.head(10)

In [ ]:
df_sample['Discounted Price']=df_sample['Sales'].apply(lambda x:0.85*x if x>200 else x)
df_sample.head(10)